# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

---
* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [3]:
# Load training data
data = spark.read.csv('/FileStore/tables/dog_food.csv',inferSchema=True,header=True)
data.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")
output = assembler.transform(data)

Out[7]: Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [5]:
final_data = output.select('features','Spoiled')
final_data.head()

Out[8]: Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [6]:
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier

rfc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
rfc_model = rfc.fit(final_data)
rfc_model.featureImportances

Out[9]: SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})

Feature at index 2 (Chemical C) is by far the most important feature, meaning it is causing the early spoilage.